# Long Short Term Memory (LSTM)

LSTM was first introduced by <a href="https://www.bioinf.jku.at/publications/older/2604.pdf">Hochreiter & Schmidhuber (1997)</a>, solving the issues of long-term dependencies that previous reccurent neural networks (RNN) had.
We will refer to the <a href="http://colah.github.io/posts/2015-08-Understanding-LSTMs/">explanatory blog post</a> for a thorough  introduction to LSTM networks.
A RNN consist of a chain of cells that are repeated throughout the network.
For a standard RNN this cell is simple and consist of only one layer with two inputs; the previous hidden state (output from the previous cell), and the new event (new data that corresponds to this timestep).
A visualization of the RNN architecture can be viewed below, where a chain of three cells is depicted.
However, this chain can be chosen to be much longer, a hyperparameter that can be tuned accordingly.

<img src="https://i.imgur.com/ZYHXrLB.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

The LSTM network has much more complex cells than a simple RNN, and it includes multiple hidden layers.
It, however, keeps the chain-like structure of repeating cells. 
In the figure below the different layers and combinatory operations are depicted. 
From the left the different layers are; *forget gate layer*, *input gate layer*, *learning layer*, and *ouput gate layer*. 
These will be further explained in the following sections. 
We will also present `numpy` implementations of the different aspects of a LSTM network as they are introduced.

<img src="https://i.imgur.com/F42ltRd.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

To start we must first implement the activation functions used in LSTM.
The first one being the sigmoid function, which will map the input values into the domain $(0, 1)$ (squeeze), and is defined by:
$$
 \sigma(x) = \frac{1}{1+e^{-x}} = \frac{e^x}{e^x+1}.
$$
Our implementation of the sigmoid function is:

In [1]:
import lstm
%psource lstm.sigmoid

def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


The second activation function is the *tanh* function, which will squeeze values between minus one and one. The function is 
$$
 \mathrm{tanh}(x) = \frac{\mathrm{sinh}(x)}{\mathrm{cosh}(x)} = \frac{e^x - e^{-x}}{e^x + e^{-x}} = \frac{e^{2x} - 1}{e^{2x} + 1},
$$
and our implementation is given below.

In [2]:
%psource lstm.tanh

def tanh(x: np.ndarray) -> np.ndarray:
    numerator = np.exp(x) - np.exp(-x)
    denominator = np.exp(x) + np.exp(-x)
    return numerator / denominator


Furthermore, in order to implement an LSTM cell in numpy, as presented in the figure above, we will create a `Parameters` class containing the entire parametrization of the LSTM network. 

In [3]:
%psource lstm.Parameters

class Parameters:
    def __init__(self, event_size: int, hidden_size: int):
        # Weights and biases for the sigmoid "f-function"
        self.event_forget_weights = np.zeros((hidden_size, event_size))
        self.event_forget_bias = 0
        self.hidden_forget_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_forget_bias = 0

        # Weights and biases for the sigmoid "i-function"
        self.event_update_weights = np.zeros((hidden_size, event_size))
        self.event_update_bias = 0
        self.hidden_update_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_update_bias = 0

        # Weights and biases for the tanh "C-bar-function"
        self.event_candidate_weights = np.zeros((hidden_size, event_size))
        self.event_candidate_bias = 0
        self.hidden_candidate_weights = np.zeros((hidden_size, hidden_size))
        self.hidden_candidate_bias = 0

        # Weights and biases for the "o-function"
        self.event_output_weight

We will now instantiate this class.
For our dummy example we will set the input size $\mathrm{dim}(x_t) = (3,1)$ and the hidden size $\mathrm{dim}(h_t) = (3,1)$.

In [4]:
EVENT_SIZE, HIDDEN_SIZE = 3, 3
parameters = lstm.Parameters(event_size=EVENT_SIZE, hidden_size=HIDDEN_SIZE)

## Cell state

In a simple RNN the hidden state, $h$, carries the information about what the network has seen in the past and will give this information to the subsequent cell.
The problem with having the hidden state carry this historical information is that each historical event is included in the loss gradient as a multiplicative factor.
This will in manny cases yeild exploding or vanishing gradients.
LSTMs introduces the *Cell State* to combat this issue.
It works by contributing to the loss function in an additive manner instead, thereby yeilding a constant gradient between current and previous time states.
The cell state is the "conveyor belt" running straight through the network and is displayed in the figure below. 

<img src="https://i.imgur.com/hyUkhnU.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

Now the hidden state, $h$, carries the information seen in the previous state, and the cell state, $C$, carries the historical data or the long-term memory.
The cell state has a multiplicative intercation with the forget gate layer, that works exactly like the name, it helps the network forget irrelevant information in the long-term memory.
It also has an additive interaction with the input/learning gate layer, that will update the long-term memory with new information to be kept for later.
Next, the cell state contributes with the calculation of the update of the hidden state in the cell.
Finally, the cell state is sent straight through to the next cell. 

## Forget Gate Layer

The *forget gate layer* is the first layer within the LSTM cell; it is furthest to the left in the cell diagram and highlighted in the visualization below.
This layer is responsible for forgetting irrelevant parts of the long-term memory kept in the cell state.
It uses the new information in $x_t$ and the information from the previous cell $h_{t-1}$ to determine this. 

<img src="https://i.imgur.com/3BwMSrF.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

$W_{hf}$ are the weights of the different units in the hidden state, $h_t$, and $W_{xf}$ are the weights of the units in the new event $x_t$.
$B_{hf}$ and $B_{xf}$ is the bias of the two different branches in the forget gate layer.
The output of these branches are added together and passed through the sigmoid activation function.
These operation can be summarized in the following equation
$$
f_t = \sigma(W_{hf}\cdot h_{t-1} + W_{xf}\cdot x_t + B_f).
$$
The output of this activation is between zero and one, and is further multiplied with the cell state.
Thereby, the cell state "forgets" irrelevant historical information. 

The forget gate is implemented as follows:

In [5]:
%psource lstm.forget_gate

def forget_gate(event, hidden_state, prev_cell_state, parameters):
    """Forget gate deciding how much of the previous cell state to keep."""
    forget_hidden = (
        parameters.hidden_forget_weights @ hidden_state
        + parameters.hidden_forget_bias
    )
    forget_event = (
        parameters.event_forget_weights @ event
        + parameters.event_forget_bias
    )
    # Values between zero and one indicating how much to forget
    forgetter = sigmoid(forget_hidden + forget_event)

    # Return the state that should be kept
    kept_state = forgetter * prev_cell_state
    return kept_state


Let's assume that we have an existing hidden state of $\vec{h} = [0, 0, 10]$, a previous cell state of $\vec{C} = [1, 1, 1]$, and a new event $\vec{x} = [10, 0, 0]$.
What happens in the forget gate with the current parametrization?

In [6]:
import numpy as np

hidden_state = np.array([0, 0, 10])
prev_cell_state = np.array([1, 1, 1])
event = np.array([10, 0, 0])
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.5, 0.5, 0.5])

Since we only have zero-weights, and zero biases, the new cell state has become $[0.5, 0.5, 0.5]$ since $\mathrm{sigmoid}(1) = 0.5$.
Let's change the weight matrices to identity matrices:

In [7]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.9999546])

Changing both weight matrices to the identity matrices, the previous hidden state and the new event has influenced the new cell state.
We can also let the new event influence the cell state, and the hidden state be completely ignored:

In [8]:
parameters.event_forget_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_forget_weights = np.zeros((HIDDEN_SIZE, EVENT_SIZE))
lstm.forget_gate(event=event, hidden_state=hidden_state, parameters=parameters, prev_cell_state=prev_cell_state)

array([0.9999546, 0.5      , 0.5      ])

We see that the first index has been changed from $0.5$ to $\approx 1$, while the third index is still $0.5$ as expected.

## Input/Learning gate Layer

The *input gate layer* is the second layer from the left in the cell diagram.
It is interacting with the *learning* layer, which is the third layer.
The figure below highlights both these layers and the operations within.
The function of these two layers is to update the cell state with new relevant information for the long-term memory.
First, the input gate decides which information from the previous cell state and the new event information is relevant (candidates).
At the same time the learning gate will weigh the different features.
Secondly, the output of these layers are multiplied together and added to the cell state.
This means that the update either "ignores" the weighted features or "remembers" them, somewhat simplified.

<img src="https://i.imgur.com/2qCZtNO.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

The input gate layer uses the sigmoid activation function. $h_{t-1}$ and $x_t$ is again weighted by individual model matricies; $W_{hi}$ and $W_{xi}$.
The bias of this layer can be combined into a single term $B_i$, and the operation can therefore be expressed as:
$$
i_t = \sigma(W_{hi}\cdot h_{t-1} W_{xi}\cdot x_t + B_i).
$$
The learning layer uses the tanh activation function.
The features $h_{t-1}$ and $x_t$ are weighted by their respective model matrices; $W_{hi}$ and $W_{xi}$, then they are added together and passed through the activation function.
These operations are summarized in the following equation:
$$
l_t = \mathrm{tanh}(W_{hl}\cdot h_{t-1} + W_{xl}\cdot x_t + B_t).
$$
$B_t$ is again the combined bias in this layer. 

Lastly, the outputs of these layers are multiplied together to form the update of the cell state.
This update is then added to the cell state, yielding the new cell state $C_t$.
This summarized in the following expression:
$$
 C_t = C_{t-1}\times f_t + i_t \times l_t. 
$$

Our implementation of the input gate and learning gate is presented below.

In [9]:
%psource lstm.input_gate

def input_gate(event, hidden_state, parameters):
    """Input gate deciding how to update the cell state."""
    # We have certain candidates from the new event and the hidden state
    # we would like to update the cell state with
    hidden_candidates = (
        parameters.hidden_candidate_weights @ hidden_state
        + parameters.hidden_candidate_bias
    )
    event_candidates = (
        parameters.event_candidate_weights @ event
        + parameters.event_candidate_bias
    )

    # We must also determine how much to weigh these updates
    event_update = (
        parameters.event_update_weights @ event
        + parameters.event_update_bias
    )
    hidden_update = (
        parameters.hidden_update_weights @ hidden_state
        + parameters.hidden_update_bias
    )

    # Finally returning the update
    return (
        sigmoid(event_update + hidden_update)
        * tanh(event_candidates + hidden_candidates)
    )


We try to evaluate this gate with the default $0$ weights.

In [10]:
lstm.input_gate(event=event, hidden_state=hidden_state, parameters=parameters)

array([0., 0., 0.])

This time we get $\vec{0}$ as the output from the input gate, due to the $\tanh$ activation function being zero for zero input.
Let's see what happens if we set the weight matrix for the hidden state to the identity matrix, but leaving the event weights as zero.

In [11]:
parameters.hidden_candidate_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
parameters.hidden_update_weights = np.eye(HIDDEN_SIZE, EVENT_SIZE)
lstm.input_gate(event=event, hidden_state=hidden_state, parameters=parameters)

array([0.       , 0.       , 0.9999546])

The resulting update weighs the hidden state and ignores the new event data completely, as expected.
The new cell state is the sum of the forget gate output and the input gate output, as follows:

In [12]:
%psource lstm.cell_state

def cell_state(forget_gate_output, input_gate_output):
    """
    New cell state, a combination of the partially forgotten cell state
    and the newly proposed state.
    """
    return forget_gate_output + input_gate_output


## Output Gate Layer

The last layer in the LSTM cell is responsible for the output of the network, referred to as the *output gate layer*.
The output is also the updated hidden state of the cell, $h_{t-1}$, which will be used in the subsequent cell.
This layer operates by chosing the information relevant from the updated cell state to the current hidden state.
The layer is highlighted in the following figure.

<img src="https://i.imgur.com/YGxPsJY.png" style="width:50%;display: block;margin-left: auto;margin-right: auto;">

Initially the cell state is applied to a tanh function to squeeze the different features between minus one and one.
It is important to note that the cell state hasn't changed in this layer, but rather used as a set of features to calculate the new hidden state.
The current hidden state $h_{t-1}$ and the event $x_t$ is then weighted according to the model matricies of the output layer; $W_{ho}$ and $W_{xo}$, and added together with a bias $B_o$.
Then the sigmoid function is applied to these weighted features.
The operations is given by the equation
$$
 o_t = \sigma(W_{ho}\cdot h_{t-1} + W_{xo}\cdot x_t + B_o).
$$
The output of this activation will choose the relevant information in the cell state by multiplying $o_t$ with the tanh of the cell state.
Thereby, $o_t$ will weigh the cell state information by their relevance to the new hidden state, and is given by the equation
$$
 h_t = o_t\times \mathrm{tanh}(C_t).
$$
The output of the LSTM is now calculated, and the new hidden state $h_t$ is sent into the next cell.
For the given LSTM network we have implemented, the event input size is $x \times 1$, and the hidden state is of size $h \times 1$.
This implies that the output of the LSTM cell becomes $h \times 1$, which is not necessarily optimal for the given problem.
It is therefore customary to pass the output into a final, fully connected neural network that will further optimize the output for the desired task.
We have opted to not implement this last part, since it is not a core, new concept of LSTM cells.

Our implementation of the output gate layer is given by the follwing code:

In [13]:
%psource lstm.output_gate

def output_gate(event, hidden_state, cell_state, parameters):
    """Decide what to output from the LSTM cell."""
    hidden_output = (
        parameters.hidden_output_weights @ hidden_state
        + parameters.hidden_output_bias
    )
    event_output = (
        parameters.event_output_weights @ event
        + parameters.event_output_bias
    )
    return (
        sigmoid(event_output + hidden_output)
        * tanh(cell_state)
    )
